<a href="https://colab.research.google.com/github/WexyR/IML_Terminator/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook info

In [1]:
import tensorflow as tf

#from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras import Model, Sequential

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
tf.__version__

'2.3.0'

In [ ]:
from google.colab import drive
drive.mount()